# OWL tutorial

First install owlready2 if you don't already have it, and have a quick look at the [documentation](https://owlready2.readthedocs.io/en/v0.36/onto.html#).

In [4]:
## Uncomment if you do not have owlrl installed (you should have it installed from the RDFS tutorial)
#import sys
#!{sys.executable} -m pip install rdflib  owlready2 pandas

import pandas as pd
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL
from rdflib.namespace import DC, FOAF

from owlready2 import *


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Let's start loading some data from a .CSV file. We are going to create an ontology that describes the data inside.
We already did part of this using the semantics of RDF(S), now we'll use the semantics of [OWL](https://www.w3.org/TR/2012/REC-owl2-primer-20121211/) through owlready2. (Tip: follow the link to avoid looking at funny pictures of owls the rest of the afternoon) 

Remember that an ontology is often an application ontology, meaning that it is built with a specific task in mind. 
We could model _everything_ within a certain domain in the most ontologically correct way possible, _or_ **we could model the domain in accordance with the application's task.** 


**Your task and domain:** You are a broadcaster that has just digitised its radio archives into a digital music archive (DMA), and aims to play more interesting tracks by discovering their 'hidden treasures', by making unexpected and potentially interesting relations between tracks visible to the users (which are journalists and program makers).


**Exercise 1** 

1. load music.csv into a pandas dataframe (use display.max_columns to show all columns). 
2. initialise an empty ontology using owlready2
3. using owlready2, create a hierarchy of classes and subclasses that describe the entities in your dataframe
4. using owrleady2, create properties and subproperties that describe how these relate to one another (using domain and range). If it helps: draw out your ontology in https://app.diagrams.net/
    - create: object properties, data properties, functional properties (you can even add a new column of data if you want) 
5. using owlready2, add class restrictions
6. create invididuals of your classes, and provide them with attributes using your properties! 
7. write simple queries to retrieve your individuals following: https://owlready2.readthedocs.io/en/v0.36/onto.html#simple-queries. What kind of things would journalists and program makers like to retrieve? 
6. save your asserted owl file

In [6]:
csv_file =  pd.read_csv('../data/musicoset_metadata/albums.csv',sep='\t')
pd.set_option('display.max_columns', None)
csv_file.head()

,album_id,name,billboard,artists,popularity,total_tracks,album_type,image_url
0,5n1GSzC1Reao29ScnpLYqp,Dying To Live,Dying To Live,{'46SHBwWsqBkxI7EeeBEQG7': 'Kodak Black'},83,16,album,https://i.scdn.co/image/db2133234d458f432ca207...
1,6UYZEYjpN1DYRW0kqFy9ZE,Championships,Championships,{'20sxb77xiYeusSH8cVdatc': 'Meek Mill'},85,19,album,https://i.scdn.co/image/77eb7c17cafe5503c58661...
2,7uVimUILdzSZG4KKKWToq0,Christmas (Deluxe Special Edition),Christmas,{'1GxkXlMwML1oSg5eLPiAz3': 'Michael Bublé'},60,20,album,https://i.scdn.co/image/2d6ee8d4fb5a45abf35cd3...
3,35s58BRTGAEWztPo9WqCIs,Spider-Man: Into the Spider-Verse (Soundtrack ...,Spider-Man: Into The Spider-Verse,{'0LyfQWJT6nXafLPZqxe9Of': 'Various Artists'},92,13,compilation,https://i.scdn.co/image/3aa37254a41cf96e815725...
4,41GuZcammIkupMPKH2OJ6I,ASTROWORLD,ASTROWORLD,{'0Y5tJX1MQlPlqiwlOH1tJY': 'Travis Scott'},91,17,album,https://i.scdn.co/image/cdca7dc20c778ada42fb18...


In [7]:
csv_file =  pd.read_csv('../data/musicoset_metadata/artists.csv',sep='\t')
pd.set_option('display.max_columns', None)
csv_file.head()

,artist_id,name,followers,popularity,artist_type,main_genre,genres,image_url
0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,34554242,96,singer,dance pop,"['dance pop', 'pop', 'post-teen pop']",https://i.scdn.co/image/b1dfbe843b0b9f54ab2e58...
1,26VFTg2z8YR0cCuwLzESi2,Halsey,7368242,90,singer,dance pop,"['dance pop', 'electropop', 'etherpop', 'indie...",https://i.scdn.co/image/22a5f3d8c42bc7cb55215e...
2,0Y5tJX1MQlPlqiwlOH1tJY,Travis Scott,6313709,94,rapper,pop,"['pop', 'pop rap', 'rap']",https://i.scdn.co/image/dc5eba5e032c2e5bc4d42c...
3,246dkjvS1zLTtiykXe5h60,Post Malone,16737002,96,rapper,dfw rap,"['dfw rap', 'pop', 'rap']",https://i.scdn.co/image/f9d8b742b66609f12da023...
4,1zNqQNIdeOUZHb8zbZRFMX,Swae Lee,483032,89,singer,trap music,['trap music'],https://i.scdn.co/image/a177469870b41f7e17e3b5...


In [8]:
csv_file =  pd.read_csv('../data/musicoset_metadata/songs.csv',sep='\t')
pd.set_option('display.max_columns', None)
csv_file.head()

,song_id,song_name,billboard,artists,popularity,explicit,song_type
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next","('Thank U, Next', 'Ariana Grande')",{'66CXWjxzNUsdJxJ2JdwvnR': 'Ariana Grande'},86,True,Solo
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,"('Without Me', 'Halsey')",{'26VFTg2z8YR0cCuwLzESi2': 'Halsey'},87,True,Solo
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,"('Sicko Mode', 'Travis Scott')",{'0Y5tJX1MQlPlqiwlOH1tJY': 'Travis Scott'},85,True,Solo
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,('Sunflower (Spider-Man: Into The Spider-Verse...,"{'246dkjvS1zLTtiykXe5h60': 'Post Malone', '1zN...",92,False,Collaboration
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,"('High Hopes', 'Panic! At The Disco')",{'20JZFwl6HVl6yg8a4H3ZqK': 'Panic! At The Disco'},86,False,Solo


In [9]:
csv_file =  pd.read_csv('../data/musicoset_metadata/tracks.csv',sep='\t')
pd.set_option('display.max_columns', None)
csv_file.head()

,song_id,album_id,track_number,release_date,release_date_precision
0,3e9HZxeyfWwjeyPAMmWSSQ,2fYhqwDWXjbpjaIJPEfKFw,11,2019-02-08,day
1,5p7ujcrUXASCNwRaWNHR1C,0zzrCTzvL4ZmR42xF46Afm,1,2018-10-04,day
2,2xLMifQCjDGFmkHkpNLD9h,41GuZcammIkupMPKH2OJ6I,3,2018-08-03,day
3,3KkXRkHbMCARz0aVfEt68P,35s58BRTGAEWztPo9WqCIs,2,2018-12-14,day
4,1rqqCSm0Qe4I9rUvWncaom,6ApYSpXF8GxZAgBTHDzYge,4,2018-06-22,day


In [10]:
onto = get_ontology("http://test.org/myonto.owl") # creates an empty ontology. Use the namespace you like!

#### Creating classes
Every class is a subclass of Thing

In [11]:
# every class is a subclass of owl:Thing!

class Person(Thing):
    namespace = onto
    
class Artist(Thing):
    namespace = onto

class Location(Thing) :
    namespace = onto
    
class Song(Thing) :
    namespace = onto
    
class Genre(Thing) :
    namespace = onto

class Member(Thing):
    namespace = onto

In [12]:
# add a class that is a subclass    
class SoloArtist(Person):
    pass 
    # no need to specify namespace here, which is derived from Person

class SubGenre(Genre):
    pass
# let's check the superclasses
print(SoloArtist.ancestors())

{myonto.Person, myonto.SoloArtist, owl.Thing}


In [13]:
# print the list of the classes in the ontology

print(list(onto.classes()))

[myonto.Person, myonto.Artist, myonto.Location, myonto.Song, myonto.Genre, myonto.Member, myonto.SoloArtist, myonto.SubGenre]


#### Create object properties
Properties of type owl:ObjectProperty have only non-literals as range, 
They are rdfs:subPropertyOf owl:topObjectProperty

In [14]:
class authorOf(ObjectProperty): 
    domain = [Artist]
    range = [Song]
    namespace = onto
    pass

class locatedIn(Artist >> Location): # another way of specifying domain and range
    namespace = onto

class hasGenre(Song >> Genre): 
    namespace = onto 
    
class writtenBy(Song >> Artist):
    namespace = onto
    
class hasFan(Artist >> Person):
    namespace = onto

class bandMember(Artist >> Member):
    namespace = onto
        

Let's add properties with restrictions too

In [15]:
# properties with restrictions
class locatedIn(ObjectProperty, TransitiveProperty): # if <A locatedin B>, and <B locatedin C>, infer <A locatedin C> 
    namespace = onto
    domain = [Location]
    range = [Location]
    
class collaboratesWith(ObjectProperty, SymmetricProperty):  #if <A collaboratesWith B>, infer <B collaboratesWith A>
    domain = [Artist]
    range = [Artist]   
    namespace = onto
    
class authorOf(Artist >> Song): #if <A authorOf B>, infer <B writtenBy A>
    inverse_property = writtenBy 
    namespace = onto
    
class isFanOf(Person >> Artist):
    inverse_property = hasFan 
    namespace = onto

#### Create datatype properties
Properties of type owl:DatatypeProperty have only literals as range,
They are rdfs:subPropertyOf owl:topDatatypeProperty

In [16]:
# datatype properties 
class followers(DataProperty, FunctionalProperty): #every Artist has a single nr of followers (per platform)
    namespace = onto
    domain = [Artist]
    range = [int] 
        
class name(DataProperty, FunctionalProperty): #every artist has a single name
    namespace = onto
    domain = [Artist]
    range = [str] 
    
class releaseDate(DataProperty, FunctionalProperty):
    namespace = onto
    domain = [Song]
    range = [str] #this can also be a datetype, but lets use str for now
    
class birthDate(DataProperty, FunctionalProperty):
    namespace = onto
    domain = [Person]
    range = [str] #this can also be a datetype, but lets use str for now
    
class height(DataProperty, FunctionalProperty):
    namespace = onto
    domain = [Person]
    range = [int] 

In [17]:
#print all properties
list(onto.properties()) # can also return .object_properties() or .data_properties()

[myonto.followers,
 myonto.name,
 myonto.releaseDate,
 myonto.birthDate,
 myonto.height,
 myonto.authorOf,
 myonto.locatedIn,
 myonto.hasGenre,
 myonto.writtenBy,
 myonto.hasFan,
 myonto.bandMember,
 myonto.collaboratesWith,
 myonto.isFanOf]

#### Adding class restrictions 
Class Restictions are special owl:Classes

In [18]:
# restriction over Artist : an artist must have at a minimum written 1 song
Artist.is_a.append(authorOf.min(1,Song)) 

# restriction over Song : a song must have at least one genre
Song.is_a.append(hasGenre.min(1))

# restriction over Song : a song must have at least one genre
Song.is_a.append(releaseDate.max(1))

# restrictions can also be specified in a class definition directly

class CollaboratingArtist(Artist):
    equivalent_to = [ Artist & collaboratesWith.some(Artist)]
    
class SoloArtist(Artist):
    equivalent_to = [Artist & bandMember.max(1)]


Let's add some disjointness between classes

In [19]:
# you cannot be an instance of an Arist and a Song at the same time 
AllDisjoint([Artist,Song])

AllDisjoint([myonto.Artist, myonto.Song])

#### Create instances (individuals)
Instances in OWL are called individuals!

In [20]:
biffy = Artist("biffy_clyro")
massive_attack = Artist("massive_attack", namespace = onto) # you can also add a namespace

# # creating individuals with properties 
tricky = SoloArtist("tricky", collaboratesWith=[massive_attack]) 

ilaria = Person("ilaria", isFanOf=[biffy], birthDate="19-03-1997" ) # because I am 23 yo
lise = Person("lise", isFanOf=[massive_attack], height=172) # and I am not as tall as I look

space = Song("Space")

# or adding a property to an individual directly
biffy.authorOf = [space]
    
uk = Location('United_Kingdom', namespace= onto)
scotland = Location("Scotland", namespace= onto, locatedIn = [uk])
edi = Location("Edinburgh", namespace= onto, locatedIn = [scotland])

biffy.locatedIn = [scotland]
massive_attack.locatedIn = [uk]

Let's also look at disjointness in our ontology

In [21]:
pop     = Genre('pop')
rock    = Genre('rock')
triphop = Genre('triphop')
experimentalRock = SubGenre('experimentalRock')

# Assert that there exist only three possible genres in this world
Genre.is_a.append(OneOf([pop, rock, triphop]))

#and that all instruments are different (instance level)
AllDifferent([pop, rock, triphop])

AllDisjoint([myonto.pop, myonto.rock, myonto.triphop])

#### Basic ontology querying

Let's search information in our ontology

In [22]:
print("Biffy's IRI : %s" % biffy.iri)
print("Ilaria is born on %s" % ilaria.birthDate)
print("")  
print("Lise's height is %s cm" % lise.height)
print("Who collaborates with someone? \n%s"% onto.search(collaboratesWith = "*"))
print("")
print("Search for a IRI containing 'ilaria'\n%s" % onto.search(iri = "*ilaria"))
print("")
print("Which artist has a fan?\n%s" % onto.search(type=onto.Artist, hasFan="*"))

## other Basic queries
# iri, for searching entities by its full IRI
# type, for searching Individuals of a given Class
# subclass_of, for searching subclasses of a given Class
# is_a*, for searching both Individuals and subclasses of a given Class

Biffy's IRI : http://test.org/myonto.owl#biffy_clyro
Ilaria is born on 19-03-1997

Lise's height is 172 cm
Who collaborates with someone? 
[myonto.tricky, myonto.massive_attack]

Search for a IRI containing 'ilaria'
[myonto.ilaria]

Which artist has a fan?
[myonto.biffy_clyro, myonto.massive_attack]


In [23]:
for d in onto.disjoints(): # will print both AllDifferent (individuals) and AllDisjoint (classes)
    print(d.entities)

[myonto.Artist, myonto.Song]
[myonto.pop, myonto.rock, myonto.triphop]


#### Save asserted triples

In [24]:
onto.save(file = "../data/my_music_ontology_asserted.owl", format = "rdfxml") # also supported ntriples

## Inference time

Let's look at how reasoning works.


Some things (eg superclasses) are already inferred by owlready2

**Exercise 2**
1. think about which things are inferred from your OWL semantics. Query/look at your graph: do you see what you expected?
2. 

In [25]:
# NB : we never asserted that tricky was a person, nor that experimentalRock was a Genre

for p in Person.instances(): # all people :
    print(p)

print("")
for i in Genre.instances(): # all genres :
    print(i) 

myonto.tricky
myonto.ilaria
myonto.lise

myonto.pop
myonto.rock
myonto.triphop
myonto.experimentalRock


In [26]:
print("Biffy has fan : %s" % biffy.hasFan) # did we specify that simon has any fan?
print("Tricky plays with : %s" % tricky.collaboratesWith) # did we specify who simon plays with?

Biffy has fan : [myonto.ilaria]
Tricky plays with : [myonto.massive_attack]


In [27]:
# this does not work! We need a reasoner ! (transitiveProperty)
print("Edinburgh is located in : %s" % edi.locatedIn) 

Edinburgh is located in : [myonto.Scotland]


In [28]:
edi

myonto.Edinburgh

### Consistency checking

Check out a full example at https://owlready2.readthedocs.io/en/latest/reasoning.html


In [32]:
# after adding all restrictions we can run a reasoner

with onto : 
    sync_reasoner(infer_property_values = True)
    # if you remove infer_property and infer_data_property, it will only infer over Classes!

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/lib/python3.10/site-packages/owlready2/hermit:/usr/local/lib/python3.10/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/ww/33zq_rh50tx94n81lb4thx0w0000gn/T/tmpby4bcheg -Y


OwlReadyJavaError: Java error message is:
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



Owlready automatically gets the results of the reasoning from HermiT (a type of reasoner) and reclassifies Individuals and Classes. 

For example, Owlready inferred 2 new superclasses for SoloArtist : MusicPerformer and Artist .

In [33]:
# let's look at its superclasses
SoloArtist.ancestors()

{myonto.Artist, myonto.Person, myonto.SoloArtist, owl.Thing}

Now we have inferred that Edinburgh is in Scotland!

In [34]:
edi.locatedIn # now it works :)

[myonto.Scotland]

#### Querying inferred triples

** Exercise 3**
Query your inferred triples: 

- *.get_parents_of(entity)* accepts any entity (Class, property or individual), and returns the superclasses (for a class), the superproperties (for a property), or the classes (for an individual). 

- *.get_instances_of(Class)* returns the individuals that are asserted as belonging to the given Class in the ontology. (NB for obtaining all instances, independently of the ontology they are asserted in, use Class.instances()).

- *.get_children_of(entity)* returns the subclasses (or subproperties) that are asserted for the given Class or property in the ontology. (NB for obtaining all children, independently of the ontology they are asserted in, use entity.subclasses()).

In [35]:
print(onto.get_parents_of(SoloArtist))
print(onto.get_children_of(Person))
print(onto.get_instances_of(Genre))

[myonto.Person, myonto.Artist]
[myonto.SoloArtist]
[myonto.pop, myonto.rock, myonto.triphop]


In [36]:
# checks for inconsistencies in the ontology
list(default_world.inconsistent_classes())

[]

#### Save inferred ontology

Once inferred all facts with can save the ontology with the asserted and inferred facts. Compare it with your ontology_asserted.owl, and check for the differences.

In [37]:
onto.save(file = "../data/my_music_ontology_inferred.owl", format = "rdfxml") # also supported ntriples